1. La información del saber TYT proviene de dataicfes
2. La información de los programas de TYT proviene de https://hecaa.mineducacion.gov.co/consultaspublicas/programas



In [1]:
##################################
# Librerías
##################################
import os
import pandas as pd
import numpy as np

In [2]:

##################################
# Constantes
##################################
pares_cines_dict = {
    "amplio": {"cine": "cine_f_2013_ac_campo_amplio", "icine": "icine_amplio"},
    "especifico": {"cine": "cine_f_2013_ac_campo_especific", "icine": "icine_spec"},
    "detallado": {"cine": "cine_f_2013_ac_campo_detallado", "icine": "icine_detall"}
}

##################################
# Funciones
##################################

def resumen_nans(df):
    resumen = pd.DataFrame({
        "column": df.columns,
        "class": [df[col].dtype for col in df.columns],
        "NA_count": [df[col].isna().sum() for col in df.columns],
        "NaN_count": [np.isnan(df[col]).sum() if pd.api.types.is_numeric_dtype(df[col]) else 0 for col in df.columns]
    })
    return resumen


def detectar_valores_no_emparejados(tabla_referencia, tabla_comparacion,
                                     columna_referencia, columna_comparacion,
                                     incluir_filas_afectadas=False):
    valores_ref = tabla_referencia[columna_referencia].dropna().unique()
    valores_comp = tabla_comparacion[columna_comparacion].dropna().unique()
    
    valores_sin_match_ref = list(set(valores_ref) - set(valores_comp))
    valores_sin_match_comp = list(set(valores_comp) - set(valores_ref))
    
    resultado = {
        "valores_sin_correspondencia_referencia": valores_sin_match_ref,
        "cantidad_valores_sin_correspondencia_referencia": len(valores_sin_match_ref),
        "valores_sin_correspondencia_comparacion": valores_sin_match_comp,
        "cantidad_valores_sin_correspondencia_comparacion": len(valores_sin_match_comp)
    }
    
    if incluir_filas_afectadas:
        resultado["filas_afectadas_referencia"] = tabla_referencia[
            tabla_referencia[columna_referencia].isin(valores_sin_match_ref)]
        resultado["filas_afectadas_comparacion"] = tabla_comparacion[
            tabla_comparacion[columna_comparacion].isin(valores_sin_match_comp)]
    
    return resultado


def resumir_por_cine(df, cine, icine, nivel_filtrado=None):
    if nivel_filtrado:
        df = df[df["nivel_de_formacion"].isin([nivel_filtrado])]
    
    resumen = df.groupby([cine, icine], dropna=False).agg(
        n_estudiantes=('punt_global', 'count'),
        promedio_punt_sabertyt=('punt_global', 'mean'),
        nombre_institucion=('nombre_institucion', 'first'),
        snies_programa=('estu_snies_prgmacademico', lambda x: "; ".join(x.dropna().astype(str).unique())),
        nombre_del_programa=('nombre_del_programa', lambda x: "; ".join(x.dropna().astype(str).unique())),
        periodo=('año_presentacion', 'first')
    ).reset_index()
    
    return resumen

def resumen_programas_con_info_cine(df, cine, nivel_filtrado=None):
    if nivel_filtrado:
        df = df[df['nivel_de_formacion'].isin([nivel_filtrado])]

    # 1️⃣ Resumen por CINE
    resumen_cine = df.groupby(cine).agg(
        promedio_cine=('punt_global', 'mean'),
        n_estudiantes_cine=('punt_global', 'count')
    ).reset_index()

    # 2️⃣ Resumen por PROGRAMA dentro del CINE
    resumen_prog = df.groupby([cine, 'estu_snies_prgmacademico']).agg(
        promedio_programa=('punt_global', 'mean'),
        codigo_institucion=('codigo_institucion', 'first'),
        nombre_institucion=('nombre_institucion', 'first'),
        nombre_del_programa=('nombre_del_programa', 'first'),
        n_estudiantes_programa=('punt_global', 'count'),
        periodo=('año_presentacion', 'first')
    ).reset_index()

    # 3️⃣ Unir ambos resúmenes para que cada programa tenga la info de su cine
    resumen_final = resumen_prog.merge(resumen_cine, on=cine, how='left')

    return resumen_final


def resumir_por_programa(df):
    resumen = df.groupby(
        ["codigo_snies_del_programa", "nombre_del_programa", "nivel_de_formacion"],
        dropna=False
    ).agg(
        n_estudiantes=('punt_global', 'count'),
        promedio_punt_sabertyt=('punt_global', 'mean'),
        n_estudiantes_matriculados=(
            'periodo',
            lambda x: sum(str(p)[-1] == "1" for p in x if pd.notnull(p))
        ),
        nombre_institucion=('nombre_institucion', 'first'),
        periodo=('año_presentacion', 'first')
    ).reset_index()
    
    return resumen



def exportar_resumenes(lista_resumenes, grupo_sufijo):
    carpeta_salida = "../../data/Promedios"
    os.makedirs(carpeta_salida, exist_ok=True)
    
    for nombre, df in lista_resumenes.items():
        filename = f"promedios_cine{nombre}_{grupo_sufijo}.csv"
        ruta_completa = os.path.join(carpeta_salida, filename)
        df.to_csv(ruta_completa, index=False)
        print(f"Exportado: {ruta_completa}")

In [3]:
##################################
# Lectura de datos
##################################
base_sabertyt = pd.read_csv("../../data/SABERTYT_cleaned/base_sbtyt_bogota_region_corte_01082025.csv")
programas_tyt = pd.read_csv("../../data/SNIES_CINE_cleaned/base_snies_tyt_bogota_region_corte_01082025.csv")

In [4]:
# Cruce de datos
data = pd.merge(
    base_sabertyt,
    programas_tyt,
    how="left",
    left_on="estu_snies_prgmacademico",
    right_on="codigo_snies_del_programa"
)


data = data[~(data["estu_snies_prgmacademico"].isna() | data["codigo_institucion"].isna())]

data["codigo_institucion"] = pd.to_numeric(data["codigo_institucion"], errors="coerce").astype("Int64")
data["codigo_snies_del_programa"] = pd.to_numeric(data["codigo_snies_del_programa"], errors="coerce").astype("Int64")


no_emparejados = detectar_valores_no_emparejados(
    base_sabertyt, programas_tyt,
    "estu_snies_prgmacademico", "codigo_snies_del_programa"
)

print(no_emparejados["cantidad_valores_sin_correspondencia_referencia"])
print(no_emparejados["cantidad_valores_sin_correspondencia_comparacion"])

411
239


In [5]:
##################################
# Resumen NaNs
##################################
data_summary = resumen_nans(data)
data_summary

,column,class,NA_count,NaN_count
0,periodo,int64,0,0
1,estu_consecutivo,object,0,0
2,estu_nivel_prgm_academico,object,0,0
3,estu_nucleo_pregrado,object,0,0
4,estu_prgm_academico,object,0,0
5,estu_prgm_codmunicipio,int64,0,0
6,estu_prgm_municipio,object,0,0
7,estu_snies_prgmacademico,float64,0,0
8,inst_cod_institucion,int64,0,0
9,inst_nombre_institucion,object,0,0


In [57]:
##################################
# Crear columna ICINE
##################################
data["icine_spec"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_especific"].astype(str)
data["icine_detall"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_detallado"].astype(str)
data["icine_amplio"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_amplio"].astype(str)

##################################
# Resumen por ICINE
##################################
años = [2020, 2021, 2022, 2023,2024]

cines_para_resumir = ["detallado"]

consolidado_tecnologico = []
consolidado_tecnicoProfesional = []

for año in años:
    data_temp = data[data["año_presentacion"] == año]
    
    resumenes_tecnologico = {
        nombre: resumir_por_cine(
            data_temp,
            pares_cines_dict[nombre]["cine"],
            pares_cines_dict[nombre]["icine"],
            "Tecnologico"
        )
        for nombre in cines_para_resumir
    }

    # Agregar los dataframes a la lista de consolidación
    for df in resumenes_tecnologico.values():
        consolidado_tecnologico.append(df)
        
    resumenes_tecnica_profesional = {
        nombre: resumir_por_cine(
            data_temp,
            pares_cines_dict[nombre]["cine"],
            pares_cines_dict[nombre]["icine"],
            "Formacion Tecnica Profesional"
        )
        for nombre in cines_para_resumir
    }

    # Agregar los dataframes a la lista de consolidación
    for df in resumenes_tecnica_profesional.values():
        consolidado_tecnicoProfesional.append(df)
    
    #exportar_resumenes(resumenes_tecnologico, f"tecnologico_{año}")
    #exportar_resumenes(resumenes_tecnica_profesional, f"tecnico_{año}")

base_tecnologico = pd.concat(consolidado_tecnologico, ignore_index=True)
base_tecnicoProfesional = pd.concat(consolidado_tecnicoProfesional, ignore_index=True)

### Pivot tables de los ICINE

In [64]:
# Paso 1: Pivotear la tabla
pivot_tecnologico = base_tecnologico.pivot_table(
    index=["icine_detall"],     # o "icine_spec", "icine_detall", según el que quieras analizar
    columns="periodo",
    values="promedio_punt_sabertyt"
)

pivot_tecnologico['NF'] = "tecnologico"

In [65]:
# Paso 1: Pivotear la tabla
pivot_tecnico_profesional = base_tecnicoProfesional.pivot_table(
    index=["icine_detall"],     # o "icine_spec", "icine_detall", según el que quieras analizar
    columns="periodo",
    values="promedio_punt_sabertyt"
)
pivot_tecnico_profesional['NF'] = "tecnico_profesional"

In [66]:
from datetime import datetime

# Generate today's date string in DDMMYYYY format
fecha_hoy = datetime.today().strftime('%d%m%Y')

pivot_tecnico_profesional.to_excel(
    f"../../data/Promedios/icine4d_tecnico_profesional_bogota_region_corte_{fecha_hoy}.xlsx", index=True
)
pivot_tecnologico.to_excel(
    f"../../data/Promedios/icine4d_tecnologico_bogota_region_corte_{fecha_hoy}.xlsx", 
    index=True
)

## Resumen por programa

In [100]:
##################################
# Resumen por programa
##################################
años = [2020, 2021, 2022, 2023,2024]
consolidado = []
for año in años:
    data_temp = data[data["año_presentacion"] == año].copy()
    promedios_programa = resumir_por_programa(data_temp)
    consolidado.append(promedios_programa)


In [101]:
base_programas = pd.concat(consolidado, ignore_index=True)

In [102]:
# Paso 1: Pivotear la tabla
pivot_base_programas = base_programas.pivot_table(
    index=["codigo_snies_del_programa","nivel_de_formacion"],     # o "icine_spec", "icine_detall", según el que quieras analizar
    columns="periodo",
    values="promedio_punt_sabertyt"
)

In [104]:
from datetime import datetime

# Generate today's date string in DDMMYYYY format
fecha_hoy = datetime.today().strftime('%d%m%Y')

pivot_base_programas.to_excel(
    f"../../data/Promedios/programas_tyt_bogota_region_corte_{fecha_hoy}.xlsx", 
    index=True
)

### Resumen por Programa con promedio de CINE y Programa

In [7]:
##################################
# Crear columna ICINE
##################################
data["icine_spec"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_especific"].astype(str)
data["icine_detall"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_detallado"].astype(str)
data["icine_amplio"] = data["codigo_institucion"].astype(str) + "_" + data["cine_f_2013_ac_campo_amplio"].astype(str)

##################################
# Resumen por ICINE
##################################
años = [2020, 2021, 2022, 2023,2024]

cines_para_resumir = ["detallado"]

consolidado_tecnologico = []
consolidado_tecnicoProfesional = []

for año in años:
    data_temp = data[data["año_presentacion"] == año]
    
    resumenes_tecnologico = {
        nombre: resumen_programas_con_info_cine(
            data_temp,
            pares_cines_dict[nombre]["cine"],
            "Tecnologico"
        )
        for nombre in cines_para_resumir
    }

    # Agregar los dataframes a la lista de consolidación
    for df in resumenes_tecnologico.values():
        consolidado_tecnologico.append(df)
        
    resumenes_tecnica_profesional = {
        nombre: resumen_programas_con_info_cine(
            data_temp,
            pares_cines_dict[nombre]["cine"],
            "Formacion Tecnica Profesional"
        )
        for nombre in cines_para_resumir
    }

    # Agregar los dataframes a la lista de consolidación
    for df in resumenes_tecnica_profesional.values():
        consolidado_tecnicoProfesional.append(df)
    
    #exportar_resumenes(resumenes_tecnologico, f"tecnologico_{año}")
    #exportar_resumenes(resumenes_tecnica_profesional, f"tecnico_{año}")

base_tecnologico = pd.concat(consolidado_tecnologico, ignore_index=True)
base_tecnicoProfesional = pd.concat(consolidado_tecnicoProfesional, ignore_index=True)

In [8]:
from datetime import datetime
column_order = [
    #'cine_f_2013_ac_campo_detallado',
    'estu_snies_prgmacademico',
    'nombre_del_programa',
    'codigo_institucion',
    'nombre_institucion',
    'n_estudiantes_programa',
    'n_estudiantes_cine',
    'promedio_programa',
    'promedio_cine',
    'periodo',
]

# Generate today's date string in DDMMYYYY format
fecha_hoy = datetime.today().strftime('%d%m%Y')

base_tecnologico.to_excel(
    f"../../data/Promedios/cine4d_tecnologico_bogota_region_{fecha_hoy}.xlsx",
    columns = column_order,
    index=False)

base_tecnicoProfesional.to_excel(
    f"../../data/Promedios/cine4d_tecnico_bogota_region_{fecha_hoy}.xlsx",
    columns = column_order,
    index=False)
